In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key="your_api_key"
)
response=llm.invoke("The first Indian to visit Space was...")
print(response.content)

Rakesh Sharma. He became the first Indian citizen to journey into outer space on April 3, 1984.


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://careers.ey.com/ey/job/Bengaluru-AIML-Senior-KA-560016/1143063101/")
page_data=loader.load().pop().page_content
print(page_data)

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
    '''
    ### SCRAPE TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website. Your job is to extract the job postings and return them in JSON format containing the following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREMBLE):
    '''
)

In [46]:
chain_extract= prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "AI/ML-Senior",
  "experience": "4+ years of working experience in Large Scale AI/ML models and data science",
  "skills": [
    "Deep understanding of statistics, AI/ML algorithms, and predictive modeling",
    "Proficiency in AI/ML programming languages like Python, R, SQL",
    "Proficiency with a deep learning framework such as TensorFlow, PyTorch or Keras",
    "Expertise in machine learning algorithms and data mining techniques (like SVM, Decision Trees, and Deep Learning Algorithms)",
    "Strong programming skills in Python including libraries for machine learning such as Scikit-learn, Pandas, NumPy, etc",
    "Utilize tools such as Docker, Kubernetes, and Git to build and manage AI pipelines",
    "Automate tasks through Python scripting, databases, and other advanced technologies like databricks, synapse, ML, AI, ADF etc",
    "Good understanding of Git, JIRA, Change / Release management, build/deploy, CI/CD Azure Devops & Share Point",
    "Strong underst

In [47]:
type(res.content)

str

In [48]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'AI/ML-Senior',
 'experience': '4+ years of working experience in Large Scale AI/ML models and data science',
 'skills': ['Deep understanding of statistics, AI/ML algorithms, and predictive modeling',
  'Proficiency in AI/ML programming languages like Python, R, SQL',
  'Proficiency with a deep learning framework such as TensorFlow, PyTorch or Keras',
  'Expertise in machine learning algorithms and data mining techniques (like SVM, Decision Trees, and Deep Learning Algorithms)',
  'Strong programming skills in Python including libraries for machine learning such as Scikit-learn, Pandas, NumPy, etc',
  'Utilize tools such as Docker, Kubernetes, and Git to build and manage AI pipelines',
  'Automate tasks through Python scripting, databases, and other advanced technologies like databricks, synapse, ML, AI, ADF etc',
  'Good understanding of Git, JIRA, Change / Release management, build/deploy, CI/CD Azure Devops & Share Point',
  'Strong understanding of Large Enterprise Applica

In [49]:
type(json_res)

dict

In [50]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
...,...,...
75,"Ruby, Spring Boot, SQLite, Vercel",https://example.com/portfolio-46
76,"Python, Flask, Oracle, Vercel",https://example.com/portfolio-47
77,"Go, React, MariaDB, GCP",https://example.com/portfolio-48
78,"C++, Flask, Redis, AWS",https://example.com/portfolio-49


In [51]:
import chromadb
import uuid

client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=[row["Techstack"]],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

In [55]:
job = json_res
job['skills']


['Deep understanding of statistics, AI/ML algorithms, and predictive modeling',
 'Proficiency in AI/ML programming languages like Python, R, SQL',
 'Proficiency with a deep learning framework such as TensorFlow, PyTorch or Keras',
 'Expertise in machine learning algorithms and data mining techniques (like SVM, Decision Trees, and Deep Learning Algorithms)',
 'Strong programming skills in Python including libraries for machine learning such as Scikit-learn, Pandas, NumPy, etc',
 'Utilize tools such as Docker, Kubernetes, and Git to build and manage AI pipelines',
 'Automate tasks through Python scripting, databases, and other advanced technologies like databricks, synapse, ML, AI, ADF etc',
 'Good understanding of Git, JIRA, Change / Release management, build/deploy, CI/CD Azure Devops & Share Point',
 'Strong understanding of Large Enterprise Applications like SAP, Oracle ERP & Microsoft Dynamics etc']

In [56]:
links = collection.query(query_texts=job['skills'], n_results=2).get('documents',[])
links

[['Machine Learning, Python, TensorFlow', 'iOS, Swift, Core Data'],
 ['Machine Learning, Python, TensorFlow', 'Python, Flask, Oracle, Vercel'],
 ['Machine Learning, Python, TensorFlow', 'iOS, Swift, ARKit'],
 ['Machine Learning, Python, TensorFlow', 'Java, Laravel, Redis, IBM Cloud'],
 ['Machine Learning, Python, TensorFlow', 'Python, Flask, Oracle, Vercel'],
 ['DevOps, Jenkins, Docker', 'C++, Angular, Firebase, IBM Cloud'],
 ['Machine Learning, Python, TensorFlow', 'Python, Django, MySQL'],
 ['DevOps, Jenkins, Docker', 'PHP, Angular, MariaDB, Azure'],
 ['PHP, Rails, Oracle, IBM Cloud', 'Ruby, Django, Redis, IBM Cloud']]

In [82]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Soham, a recent graduate from IIITN (an Institute of National Importance renowned for producing top-tier professionals in technology and engineering) with a degree in Electronics and Communication Engineering. 
    You have a strong foundation in software engineering, artificial intelligence, and data science, gained through academic coursework and hands-on experience in projects, internships, and hackathons. These experiences have honed your problem-solving skills and prepared you to contribute meaningfully to any organization.
    
    Write a professional and concise cold email to the HR manager that is visually easy to read, engaging, and tailored to the job description provided above. Ensure the email includes the following elements:
    - **Subject Line**: A straightforward subject line such as “Application for [Job Title] Role”.
    - **Opening Statement**: Start with a polite and enthusiastic introduction expressing interest in the role.
    - **Key Skills and Achievements**: Briefly highlight the most relevant skills, accomplishments, and experiences that align with the job requirements.
    - **Portfolio**: Include the most relevant ones from the following links to showcase your portfolio: {link_list}.
    - **Alignment with the Company**: Explain how your background and skills align with the company’s goals and how you can drive its growth.
    - **Call to Action**: End with a clear and professional call to action (e.g., requesting an opportunity to discuss further or schedule an interview).
    - **Closing**: Use a polite and professional closing line with your full name.
    
    The email should:
    - Be formatted for easy skimming with short paragraphs or bullet points where appropriate.
    - Avoid unnecessary preambles or irrelevant details.
    - Maintain a professional tone while conveying enthusiasm and motivation.

    ### EMAIL:
    """
)


In [83]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list":links})
print(res.content)

Subject: Application for AI/ML-Senior Role

Dear Hiring Manager,

I am writing to express my enthusiastic interest in the AI/ML-Senior position at your esteemed organization. As a recent graduate from IIITN with a degree in Electronics and Communication Engineering, I am confident that my strong foundation in software engineering, artificial intelligence, and data science makes me an ideal candidate for this role.

**Key Skills and Achievements:**

* Proficient in AI/ML programming languages like Python, with expertise in deep learning frameworks such as TensorFlow and PyTorch
* Strong understanding of machine learning algorithms and data mining techniques, including SVM, Decision Trees, and Deep Learning Algorithms
* Experienced in utilizing tools such as Docker, Kubernetes, and Git to build and manage AI pipelines
* Skilled in automating tasks through Python scripting, databases, and other advanced technologies like Databricks and Azure DevOps
* Familiarity with Large Enterprise Appl